In [1]:
# Setup the Jupyter version of Dash
from dash import Dash
from dash import dcc, html


# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from bson import ObjectId
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD module for accessing the Animal Shelter database
from CRUDMongo_2 import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Setup MongoDB connection credentials
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
CRUDMongo_2 = AnimalShelter(username, password)

# load file into a pandas Dataframe
df = pd.read_csv("aac_shelter_outcomes.csv")

# Only insert the CSV data into MongoDB if the collection is currently empty
if len(list(CRUDMongo_2.read({}))) == 0:
    df = pd.read_csv("aac_shelter_outcomes.csv")
    data_to_insert = df.to_dict(orient='records')
    for record in data_to_insert:
        CRUDMongo_2.create(record)
    print("Data inserted.")
else:
    print("Data already exists in MongoDB.")

# Fetch all data from MongoDB as a test
data_from_db = CRUDMongo_2.read({})

# Ensure that the '_id' column exists before trying to drop it
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Joseph Szabo, CS-340 Dashboard'))),
    html.Hr(),
    #html.Div(
        
    # Radio Buttons for the filtering options.
    dcc.RadioItems(
        id='filter-type', 
        options=[
            {'label': 'All Dogs', 'value': 'RESET'}, 
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster/Individual Tracking', 'value': 'DIT'},
        ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
    ),

    # Radio items for selecting chart type
    dcc.RadioItems(
    id='chart-selector',
    options=[
        {'label': 'Breed Distribution', 'value': 'pie_chart'},
        {'label': 'Outcome Distribution', 'value': 'bar_chart'},
        {'label': 'Days in Shelter Distribution', 'value': 'histogram'}, 
    ],
    value='pie_chart',
    labelStyle={'display': 'inline-block'}
    ),

    html.Hr(),

    # Data table to display the animal shelter data
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable = "single", # allows a row to be selected
        editable=True,
        selected_rows=[0],
        page_action="native", # enable native page transition
        sort_action="native", # Allow for sorting
        filter_action="native", # Allow filter options
        page_current=0, #set the start page = 0
        page_size=10, # Set 10 rows per page                 
    ),
    
    html.Br(),
    html.Hr(),
    
    # Sise-by-side graphs and geolocation map
    html.Div(className='row',style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the table based on the selected filter
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Load all the data by default
    df = pd.DataFrame.from_records(CRUDMongo_2.read({}))

    # Apply filtering based on the selected filter type
    if filter_type == 'RESET':
        # Filter to only show 'Dog' entries (if no specific filter is selected)
        df = pd.DataFrame.from_records(CRUDMongo_2.read({'animal_type': 'Dog'}))
    
    elif filter_type == 'WR':  # Water Rescue filter
        df = pd.DataFrame(list(CRUDMongo_2.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},  # Water Rescue requires Intact Female
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland Mix'},
                    {'breed': 'Newfoundland Australian Cattle Dog'},
                    {'breed': 'Newfoundland/Great Pyrenees'},
                    {'breed': 'Newfoundland/Labrador Retriever'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},  # Age filtering for Water Rescue
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))

    elif filter_type == 'MWR':  # Mountain/Wilderness Rescue filter
        df = pd.DataFrame(list(CRUDMongo_2.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},  # Mountain/Wilderness Rescue requires Intact Male
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},  # Age filtering for Mountain/Wilderness Rescue
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))

    elif filter_type == 'DIT':  # Disaster/Individual Tracking filter
        df = pd.DataFrame(list(CRUDMongo_2.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},  # Disaster/Individual Tracking requires Intact Male
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 20}},  # Age filtering for Disaster/Individual Tracking
                    {'age_upon_outcome_in_weeks': {'$lte': 300}}
                ]}
            ]
        })))

    # Ensure datetime columns are processed
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])  # Convert datetime to proper format
        df['intake_date'] = df['datetime'] - pd.to_timedelta(np.random.randint(10, 61, size=len(df)), unit='d')  # Add random intake date
        df['days_in_shelter'] = (df['datetime'] - df['intake_date']).dt.days  # Calculate days in shelter

    # Convert '_id' to string if it exists
    if '_id' in df.columns:
        df = df.drop(columns=['_id'])

    # Prepare data for the table
    data = df.to_dict('records')
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    return data, columns


#Call back to display the chart based on the selected chart type
@app.callback(
    Output('graph-id', 'children'),
    [Input('chart-selector', 'value'),
     Input('datatable-id', 'derived_virtual_data')]
)
def update_graph(selected_chart, viewData):
    if viewData is None or len(viewData) == 0:
        return []

    # Convert viewData into a pandas DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Display different charts based on selected value
    if selected_chart == 'pie_chart':
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Preferred Animals')
            )
        ]
    
    elif selected_chart == 'bar_chart':
        if 'outcome_type' in dff.columns and not dff['outcome_type'].isnull().all():
            return [
                dcc.Graph(
                    figure=px.bar(dff, x='outcome_type', title='Outcome Type Distribution')
                )
            ]
    
    elif selected_chart == 'histogram':
        return [
            dcc.Graph(
                figure=px.histogram(dff, x='days_in_shelter', nbins=20, title='Days in Shelter Distribution')
            )
        ]
    
    return []

    # Only show dogs when filter Reset
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(CRUDMongo_2.read({'animal_type': 'Dog'}))

        # Convert to datetime format and calculate new columns
        df['datetime'] = pd.to_datetime(df['datetime'])
        df['intake_date'] = df['datetime'] - pd.to_timedelta(np.random.randint(10, 61, size=len(df)), unit='d')
        df['days_in_shelter'] = (df['datetime'] - df['intake_date']).dt.days

        #debugging print some columns to check
        print(df[['datetime', 'intake_date', 'days_in_shelter']].head(10))

        #prepare data for display
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        # convert ObjectIds to strings 
        data = [convert_objectid_to_str(record) for record in data]

        print(dff.columns) # debuggin output

        return data,columns
                 
    # Water Rescue Filter
    elif filter_type == 'WR':
        df = pd.DataFrame(list(CRUDMongo_2.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland Mix'},
                    {'breed': 'Newfoundland Australian Cattle Dog'},
                    {'breed': 'Newfoundland/Great Pyrenees'},
                    {'breed': 'Newfoundland/Labrador Retriever'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))

        # Convert to datetime format and calculate new columns
        df['datetime'] = pd.to_datetime(df['datetime'])
        df['intake_date'] = df['datetime'] - pd.to_timedelta(np.random.randint(10, 61, size=len(df)), unit='d')
        df['days_in_shelter'] = (df['datetime'] - df['intake_date']).dt.days

        #prepare data for display
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        #convert ObjectIds to strings 
        data = [convert_objectid_to_str(record) for record in data]
        return data,columns
        
    # Mountain/Wilderness Rescue Filter
    elif filter_type == 'MWR':
        df = pd.DataFrame(list(CRUDMongo_2.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))

        # Convert to datetime format and calculate new columns
        df['datetime'] = pd.to_datetime(df['datetime'])
        df['intake_date'] = df['datetime'] - pd.to_timedelta(np.random.randint(10, 61, size=len(df)), unit='d')
        df['days_in_shelter'] = (df['datetime'] - df['intake_date']).dt.days

        #prepare data for display
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        #convert ObjectIds to strings
        data = [convert_objectid_to_str(record) for record in data]
        return data,columns
        
    # Disaster/Individual tracking filter
    elif filter_type == 'DIT':
        df = pd.DataFrame(list(CRUDMongo_2.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 20}},
                    {'age_upon_outcome_in_weeks': {'$lte': 300}}
                ]}
            ]
        })))

        # Convert to datetime format and calculate new columns
        df['datetime'] = pd.to_datetime(df['datetime'])#################################################################
        df['intake_date'] = df['datetime'] - pd.to_timedelta(np.random.randint(10, 61, size=len(df)), unit='d')######## new columnmsn
        df['days_in_shelter'] = (df['datetime'] - df['intake_date']).dt.days##########################################

        #debugging
        print(df[['datetime', 'intake_date', 'days_in_shelter']].head(10))

        # prepare data for display
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        # convert ObjectIds to strings
        data = [convert_objectid_to_str(record) for record in data]
        return data,columns
    
    #If no condition matches
    return [], []

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    #Check if selected_columns are empty
    if selected_columns is None or len(selected_columns) == 0:
        return []
    
    # Highlight selected column
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Return default map if no data or selection
    if not viewData or not index:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])
    
    try:
        dff = pd.DataFrame.from_dict(viewData)
        if '_id' in dff.columns:
            dff = dff.drop(columns=['_id'])

        # Use selected row index or fallback to first row
        row = index[0] if index else 0

        # Use column names instead of column numbers
        lat = float(dff.iloc[row]['location_lat'])
        lon = float(dff.iloc[row]['location_long'])

        name = dff.iloc[row].get('name', 'Unknown')
        breed = dff.iloc[row].get('breed', 'Unknown')

        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(name),
                    dl.Popup([
                        html.H4("Animal Info"),
                        html.P(f"Name: {name}"),
                        html.P(f"Breed: {breed}")
                    ])
                ])
            ])
        ]
    except Exception as e:
        print(f"Map error: {e}")  #debugging
        # Return basic map fallback
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])

Connected successfully to MongoDB!
Data already exists in MongoDB.


In [2]:
app.run(debug=True)